In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import KFold
from scipy import stats
from tensorflow.keras.applications.resnet50 import ResNet50


In [ ]:
def create_training_samples(image_folder, annotation_folder, target_size):
    training_samples = []

    for image_file in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_file)
        annotation_path = os.path.join(annotation_folder, image_file.replace(".jpg", ".txt"))
        real_img = cv2.imread(image_path)
        img = cv2.cvtColor(real_img, cv2.COLOR_BGR2RGB)

        with open(annotation_path, 'r') as annotation_file:
            lines = annotation_file.readlines()

            for line in lines:
                data = line.strip().split()
                class_label = int(data[0])
                x_center, y_center, width, height = map(float, data[1:5])


                # Convert YOLO format to (x, y, w, h)
                x = int((x_center - width / 2) * img.shape[1])
                y = int((y_center - height / 2) * img.shape[0])
                w = int(width * img.shape[1])
                h = int(height * img.shape[0])

                # Read the image and crop the region defined by the bounding box

                # cropped_region = img[y:y+h, x:x+w]
                cropped_region = img[max(y, 0):min(y+h, img.shape[0]), max(x, 0):min(x+w, img.shape[1])]

                # Resize the cropped region to the target size
                resized_cropped_region = cv2.resize(cropped_region, target_size)

                # Normalize pixel values to [0, 1]
                normalized_cropped_region = resized_cropped_region / 255.0

                # Append the training sample along with its class label
                training_samples.append((normalized_cropped_region, class_label))

    return training_samples

In [ ]:
image_folder = '/kaggle/input/paddy-feild/resized images'
annotation_folder = '/kaggle/input/paddy-feild/labels'
target_size = (224, 224)
training_samples = create_training_samples(image_folder, annotation_folder, target_size)

In [ ]:
# Convert to numpy arrays
X = np.array([sample[0] for sample in training_samples])
y = np.array([sample[1] for sample in training_samples])

# Initialize k-fold cross-validation
k = 7
kf = KFold(n_splits=k, shuffle=True, random_state=42)


# Lists to store metrics
train_accuracies, train_precisions, train_recalls, train_aucs, train_f1_scores = [], [], [], [], []
val_accuracies, val_precisions, val_recalls, val_aucs, val_f1_scores = [], [], [], [], []

In [ ]:
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\nFold {fold+1}/{k}")
    
    # Data split
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Model definition
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)

    resnet = Model(inputs=base_model.input, outputs=predictions)
    resnet.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall', 'auc']
    )

    # Callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)

    # Train model
    history = resnet.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=30,
        batch_size=16,
        callbacks=[early_stop, reduce_lr],
        verbose=1
    )

    # ---- Training metrics from last epoch ----
    h = history.history
    train_accuracy = h['accuracy'][-1]
    train_precision = h['precision'][-1]
    train_recall = h['recall'][-1]
    train_auc = h['auc'][-1]
    train_f1 = 2 * (train_precision * train_recall) / (train_precision + train_recall) if (train_precision + train_recall) > 0 else 0.0

    train_accuracies.append(train_accuracy)
    train_precisions.append(train_precision)
    train_recalls.append(train_recall)
    train_aucs.append(train_auc)
    train_f1_scores.append(train_f1)

    # ---- Validation metrics ----
    
    val_accuracy = h['val_accuracy'][-1]
    val_precision = h['val_precision'][-1]
    val_recall = h['val_recall'][-1]
    val_auc = h['val_auc'][-1]
    val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall) if (val_precision + val_recall) > 0 else 0.0

    val_accuracies.append(val_accuracy)
    val_precisions.append(val_precision)
    val_recalls.append(val_recall)
    val_aucs.append(val_auc)
    val_f1_scores.append(val_f1)

    # ---- Print Results ----
    print(f"\nFold {fold+1} Training Metrics:")
    print(f"Train Accuracy : {train_accuracy:.4f}")
    print(f"Train Precision: {train_precision:.4f}")
    print(f"Train Recall   : {train_recall:.4f}")
    print(f"Train F1-Score : {train_f1:.4f}")
    print(f"Train AUC      : {train_auc:.4f}")

    print(f"\nFold {fold+1} Validation Metrics:")
    print(f"Val Accuracy   : {val_accuracy:.4f}")
    print(f"Val Precision  : {val_precision:.4f}")
    print(f"Val Recall     : {val_recall:.4f}")
    print(f"Val F1-Score   : {val_f1:.4f}")
    print(f"Val AUC        : {val_auc:.4f}")

In [ ]:
print("\nTraining Metrics Summary:")
print(f"Train Accuracy : {train_accuracies}")
print(f"Train Precision: {train_precisions}")
print(f"Train Recall   : {train_recalls}") 
print(f"Train F1-Score : {train_f1_scores}")
print(f"Train AUC      : {train_aucs}")
print(f"\nValidation Metrics Summary:")
print(f"Val Accuracy   : {val_accuracies}")
print(f"Val Precision  : {val_precisions}")
print(f"Val Recall     : {val_recalls}")
print(f"Val F1-Score   : {val_f1_scores}")
print(f"Val AUC        : {val_aucs}")